In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt
import dask
import dask.dataframe as dd

In [3]:
#my subset
df1 = dd.read_csv('accepted_2007_to_2018Q4.csv', usecols=['acc_now_delinq','acc_open_past_24mths','addr_state','all_util',
                'annual_inc','annual_inc_joint','application_type','avg_cur_bal',
                'bc_open_to_buy','bc_util','chargeoff_within_12_mths','collection_recovery_fee',
                'collections_12_mths_ex_med','debt_settlement_flag','debt_settlement_flag_date',
                'deferral_term','delinq_2yrs','delinq_amnt','disbursement_method',
                'dti','dti_joint','earliest_cr_line','emp_length','emp_title',
                'fico_range_high','fico_range_low','funded_amnt','funded_amnt_inv',
                'grade','int_rate','loan_status','issue_d'], low_memory = False)

In [4]:
def recent_years(df):
    df['issue_d']=dd.to_datetime(df.issue_d)
    df = df.loc[df['issue_d'].dt.year.between(2015,2018)]
    df = df.dropna(subset=['issue_d'])
    return df 

In [5]:
def grade_change(df):
    #drop F & G
    df = df[(df.grade != 'F') & (df.grade != 'G')]
    #encode grades
    df['grade'] = df['grade'].mask(df['grade'] == 'E', '1').mask(df['grade'] == 'D', '2').mask(df['grade'] == 'C', '3').mask(df['grade'] == 'B', '4').mask(df['grade'] == 'A', '5')
    return df

In [7]:
def annual_inc(df):
    df['annual_inc_total'] = df['annual_inc_joint'].fillna(df['annual_inc'])
    #drop 2 outliers
    df = df[df.annual_inc_total < .2e8]
    df = df.drop(['annual_inc','annual_inc_joint'], axis=1)
    return df

In [8]:
# convert columns to binary
import dask.array as da

def make_binary(df):
    x = df1['application_type'].to_dask_array()
    y = df1['disbursement_method'].to_dask_array()
    df['application_type_indiv']= da.where(x =='Individual',1,0)
    df['disbursement_method_cash']= da.where(y =='Cash',1,0)
    df = df.drop(['application_type','disbursement_method'], axis=1)
    return df

In [9]:
def date_format(df):
    df['earliest_cr_line']=dd.to_datetime(df.earliest_cr_line,unit='ns')
    return df

In [ ]:
def drop_columns(df):
    df = df.drop(['desc','collection_recovery_fee','debt_settlement_flag','debt_settlement_flag_date','deferral_term','funded_amnt','funded_amnt_inv'], axis=1)
    return df

#### Still need to deal with:

In [ ]:
# addr_state needs to be dummified

Missingness: 
- Mean impute all_util - 388,531 missing

In [16]:
df1.all_util = df1.all_util.fillna(np.mean(df1.all_util))

- avg_cur_bal - 55 missing

In [ ]:
#drop?
df = df.dropna(subset=['avg_cur_bal'])
#mean impute?
df1.avg_cur_bal = df1.avg_cur_bal.fillna(np.mean(df1.avg_cur_bal))

- bc_open_to_buy - 20,182 missing
- bc_util - 21,034 missing


- dti - 1666 missing - verification status? (is annual_inc ok?)
- dti_joint - 1,638,776 missing - verification status?


- emp_length - missing 123,534
- emp_title - ???